# RMSNorm
核心思想：
RMSNorm (Root Mean Square Normalization) 是一种归一化技术，其主要思想是对输入张量的每个元素除以其均方根 (RMS)，以达到稳定神经网络训练的目的。与常见的 Layer Normalization 不同，RMSNorm 不会减去均值，这意味着它保留了特征的“平移不变性”（re-centering invariance），更侧重于对特征的**幅度（magnitude）**进行归一化。</br>
1. 计算均方根
2. 归一化：也就是除以均方根
3. （可选）对张量进行缩放，该参数weight可被学习


In [3]:
import torch
import torch.nn as nn

In [4]:
class RMSNorm(nn.Module):
  def __init__(self, dim, eps = 1e-6):
    super().__init__()
    self.eps = eps
    self.dim = dim
    self.weight = nn.Parameter(torch.ones(dim))


  def _norm(self, x):
    rms_x = torch.rsqrt(x.pow(2).mean(-1, keepdim = True) + self.eps)
    return x * rms_x


  def forward(self, x):
    return self._norm(x.float()).type_as(x) * self.weight

In [5]:
# --- 验证 RMSNorm 的正确性 ---
def check_rms_norm():
    # 创建一个随机输入张量
    batch_size = 2
    seq_len = 5
    feature_dim = 4
    x = torch.randn(batch_size, seq_len, feature_dim)

    # 初始化 RMSNorm 层
    rms_norm_layer = RMSNorm(dim=feature_dim)

    # 进行前向传播
    output = rms_norm_layer(x)

    print("--- RMSNorm 验证 ---")
    print(f"输入张量 x 形状: {x.shape}")
    print(f"输入张量 x 数据类型: {x.dtype}")
    print(f"输入张量 x 设备: {x.device}")
    print("\n原始输入 x:")
    print(x)

    print("\nRMSNorm 归一化后的输出:")
    print(output)
    print(f"输出张量 output 形状: {output.shape}")
    print(f"输出张量 output 数据类型: {output.dtype}")
    print(f"输出张量 output 设备: {output.device}")

    # 验证归一化效果：
    # 计算输出的均方根，应该接近 1 (在考虑 gamma 缩放后)。
    # 如果 gamma 是全1，那么每个特征向量的 RMS 应该接近 1。
    # 我们可以手动计算一下：
    # 移除 gamma 的影响，即 output / weight
    output_without_gamma = output / rms_norm_layer.weight
    rms_of_output = torch.sqrt((output_without_gamma.pow(2).mean(-1, keepdim=True)))
    print("\n每个归一化后特征向量的 RMS (不含gamma):")
    print(rms_of_output)
    # 期望这些值都接近 1.0

    # 也可以测试混合精度
    x_fp16 = x.half() # 转换为 float16
    output_fp16 = rms_norm_layer(x_fp16)
    print("\n--- 混合精度 (float16) 测试 ---")
    print(f"输入张量 x_fp16 数据类型: {x_fp16.dtype}")
    print(f"输出张量 output_fp16 数据类型: {output_fp16.dtype}")
    print(f"输出张量 output_fp16 形状: {output_fp16.shape}")
    print("\nRMSNorm 归一化后的 float16 输出:")
    print(output_fp16)

In [6]:
check_rms_norm()

--- RMSNorm 验证 ---
输入张量 x 形状: torch.Size([2, 5, 4])
输入张量 x 数据类型: torch.float32
输入张量 x 设备: cpu

原始输入 x:
tensor([[[ 1.3819, -0.8070, -1.9535,  1.4018],
         [ 0.0659,  1.1150,  0.1526,  0.0918],
         [ 0.0055, -0.4674,  0.0471, -0.1710],
         [ 1.6314,  0.0165,  0.1086,  1.0166],
         [ 0.3424, -0.0889,  0.4060, -0.8189]],

        [[-0.6987, -0.4687, -0.7417, -0.2912],
         [ 0.3796, -0.9872, -1.0510,  1.3867],
         [-1.4448, -1.2028,  0.6512, -0.4053],
         [-2.1989, -1.0851, -2.0173, -0.0086],
         [-0.1549, -0.0707,  0.7981,  0.2502]]])

RMSNorm 归一化后的输出:
tensor([[[ 0.9569, -0.5588, -1.3527,  0.9707],
         [ 0.1165,  1.9716,  0.2698,  0.1624],
         [ 0.0221, -1.8698,  0.1885, -0.6839],
         [ 1.6946,  0.0172,  0.1128,  1.0560],
         [ 0.6986, -0.1813,  0.8285, -1.6711]],

        [[-1.2059, -0.8090, -1.2801, -0.5026],
         [ 0.3728, -0.9696, -1.0323,  1.3620],
         [-1.4232, -1.1848,  0.6415, -0.3993],
         [-1.3850, -0.6835,